In [67]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

import nltk   
nltk.download('punkt')
nltk.download('stopwords')
from nltk.corpus import stopwords, wordnet
from nltk import PorterStemmer, WordNetLemmatizer 

import string
import collections

from sklearn.cross_validation import train_test_split
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer

from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from xgboost.sklearn import XGBClassifier

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Yunsun2\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Yunsun2\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [68]:
data=pd.read_csv('C:/Users/Yunsun2/Desktop/Github/yelp_cleaned.csv', encoding='latin-1', index_col=0)

In [69]:
#Sentiment analysis;
def pos_score(text):
    pos_words={'ambiance', 'ambience', 'awhile', 'delight', 'generous', 'hilarious', 'homey', 'hometown', 'home'
               'performance', 'performer', 'recover', 'tremendous', 'fantastically', 'fantastic', 'friendly', 'extraordinarily' 
               'goodwill', 'successfully', 'amaze', 'amazing', 'amazingly', 'magic', 'favourite', 'favorite', 'fresh', 
               'great', 'love', 'best', 'delicious', 'perfect', 'awesome', 'recommend', 'excellent', 'highly', 'incredible'
               'friendly', 'wonderful', 'perfectly', 'beautiful', 'fabulous', 'cool', 'super', 'heaven'} 
    pos_score=0
    for i in str(text).split():
        if i in pos_words:
            pos_score+=1
    return pos_score
    
def neg_score(text):
    neg_words={'awkwardly', 'awkward', 'butt', 'discount', 'disappointed', 'disappoint', 
               'dismay', 'dumb', 'manner', 'sue', 'mean', 'meeting', 'monte', 'unfriendly', 'nasty', 'refuse', 
               'avoid', 'soggy', 'mistake', 'upset', 'filthy', 'sad', 'lousy', 'worst',  'unprofessional', 
               'rudely','unfortunately', 'screw', 'poorly', 'crap', 'crappy', 'tasteless', 'ignore', 'disaster',
               'overpriced', 'wrong', 'sick', 'inedible', 'stale', 'dirty', 'disappointment', 'sorry', 'yuck', 
               'flavorless', 'mess', 'dont', 'slow', 'lack', 'wouldnt', 'disappointed', 'terrible', 'overprice', 
               'gross', 'waste', 'poor', 'leave', 'disgust', 'awful', 'mediocre', 'suck', 'bland', 'wasnt', 
               'rude', 'didnt', 'horrible', 'bad'}
    neg_score=0
    for i in str(text).split():
        if i in neg_words:
            neg_score-=1
    return neg_score
         
data['pos_score']=data['textc'].apply(pos_score)
data['neg_score']=data['textc'].apply(neg_score)
#print(data.corr())  #sentiment now is 0.49 corr with stars; 
#if change to pos_score & neg_score, now pos is 0.346 cor, and neg_score is 0.402 cor

In [70]:
data['stars'].value_counts()

4    3526
5    3337
3    1461
2     927
1     749
Name: stars, dtype: int64

In [82]:
#re-value stars 4 & 5
data['star']=data['stars'].apply(lambda x: 5 if x>=4 else x)
data.drop('stars', axis=1, inplace=True)

In [79]:
data['year']=data['date'].apply(lambda x: x.split('-')[0])
data['month']=data['date'].apply(lambda x: x.split('-')[1])
data.head()

,business_id,date,review_id,stars,text,type,user_id,cool,useful,funny,textc,text_len,pos_score,neg_score,year,month
0,9yKzy9PApeiPPOUJEtnvkg,2011-01-26,fWKvX83p0-ka4JS3dc6E5A,5,My wife took me here on my birthday for breakf...,review,rLtl8ZkDX5vH5nAx9C3q5Q,2,5,0,wife birthday breakfast excellent weather perf...,445,10,0,2011,01
1,ZRJwVLyzEJq1VAihDhYiow,2011-07-27,IjZ33sJrzXqU-0X6U8NwyA,5,I have no idea why some people give bad review...,review,0a2KyEL0d3Yb1V6aivbIuQ,0,0,0,no idea why bad show please everyone probably ...,545,2,-3,2011,07
2,6oRAC4uyJCsJl1X0WZpVSA,2012-06-14,IESLBzqUCLdSzSqm0eCSxQ,4,love the gyro plate. Rice is so good and I als...,review,0hT2KtfLiobPvh6cDC8JQg,0,1,0,love gyro plate rice so good dig candy selection,48,1,0,2012,06
3,_1QQZuf4zZOyFCvXc0o6Vg,2010-05-27,G-WvGaISbqqaMHlNnByodA,5,"Rosie, Dakota, and I LOVE Chaparral Dog Park!!...",review,uZetl9T0NcROGOyFfughhg,1,2,0,love chaparral dog park very convenient surrou...,198,2,0,2010,05
4,6ozycU1RpktNG2-1BroVtw,2012-01-05,1uJFq2r5QfJG_6ExMRCaGw,5,General Manager Scott Petello is a good egg!!!...,review,vYmM4KTsC8ZfQBg-j5MWkw,0,0,0,general manager scott good egg not into detail...,256,2,-2,2012,01


In [83]:
from sklearn.cross_validation import train_test_split
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer

Z=data.drop('textc', axis=1)
X=data['textc'] #will add other features back later on for another analysis;   

rs=3746
x_train, x_test, ztrain, ztest = train_test_split(X, Z, test_size=0.3, random_state=rs) 

#TfidfVectorizer
tfidfv=TfidfVectorizer(min_df=2, ngram_range=(1, 1), stop_words='english', max_features=10000, strip_accents='unicode', 
                           norm='l2')

xtrain=tfidfv.fit_transform(x_train.values.astype('U')).todense()
xtest=tfidfv.transform(x_test.values.astype('U')).todense()
col = ['feat_'+i for i in tfidfv.get_feature_names()]  #for TfidfVectorizer

_xtrain = pd.DataFrame(xtrain, columns=col)  #feature names added
_xtest = pd.DataFrame(xtest, columns=col)  #row index from 0 to 6999 (number of reviews);  

temptrain=ztrain.loc[:, ['cool', 'useful', 'funny', 'text_len', 'pos_score', 'neg_score', 'year', 'month']].reset_index(drop=True)  #remove 'sentiment' needed;
trainfull=pd.concat([temptrain, _xtrain], axis=1)

temptest=ztest.loc[:, ['cool', 'useful', 'funny', 'text_len', 'pos_score', 'neg_score', 'year', 'month']].reset_index(drop=True) #remove 'sentiment' needed;
testfull=pd.concat([temptest, _xtest], axis=1)

ytrain=ztrain['star']
ytest=ztest['star']

In [84]:
from sklearn.ensemble import RandomForestRegressor

result=pd.DataFrame()
rs=1928
global result

'''
model=RandomForestRegressor(random_state=rs)
cname='RandomForestRegressor'

model=XGBClassifier(random_state=rs)
cname='XGBoost'
'''
model=LogisticRegression(random_state=rs)
cname='LogisticRegression'

model.fit(trainfull, ytrain)
pred=model.predict(testfull)
tpred=model.predict(trainfull)
    
#RFRegressor has not attributes of predict_proba; 
predproba=pd.DataFrame(model.predict_proba(testfull))
tpredproba=pd.DataFrame(model.predict_proba(trainfull))
#merge with test and train labels; 

print(cname + ' predictions with default hyparameters for Test'+'\n')
print('Accuracy')
print(accuracy_score(ytest, pred, normalize=True))
print(confusion_matrix(ytest, pred))
print(classification_report(ytest, pred))  

print(cname + 'predictions with default hyparameters for Train'+'\n')
print('Accuracy')
print(accuracy_score(ytrain, tpred, normalize=True))

#compare bad predictions of different models; 
result=pd.concat([result, pd.DataFrame(pred)], axis=1) #to merge the predicted labels back to the dataset;
result.rename(columns={0:str(cname + '_pred')}, inplace=True)

LogisticRegression predictions with default hyparameters for Test

Accuracy
0.72
[[  83   31    4  111]
 [  32   52   12  173]
 [  18   31   33  395]
 [   8   10   15 1992]]
             precision    recall  f1-score   support

          1       0.59      0.36      0.45       229
          2       0.42      0.19      0.26       269
          3       0.52      0.07      0.12       477
          5       0.75      0.98      0.85      2025

avg / total       0.67      0.72      0.65      3000

LogisticRegressionpredictions with default hyparameters for Train

Accuracy
0.781571428571


In [ ]:
#Much better now! Star 2 improved due to pos_ and neg_ sentiment added.
#After adding year and month, star 3, 5 improved, while others slightly droped. 
#If cared about stars 4 and 5, then better to group them into 1 group. Very hard to differentiate. 